In [10]:
import pandas as pd
import numpy as np
suffix=f'先进制造未上市'
doc_lda_w2v_1=pd.read_parquet(f'data\\documents_vec_{suffix}.parquet')  

suffix=f'先进制造上市'
doc_lda_w2v_2=pd.read_parquet(f'data\\documents_vec_{suffix}.parquet')

doc_lda_w2v_1=pd.concat([doc_lda_w2v_1,doc_lda_w2v_2])

patent_co=pd.read_csv('data\patent_co_1953_16k_202407.csv',index_col=0)
patent_co['申请日']=pd.to_datetime(patent_co['申请日'])
patent_co_1=patent_co[(patent_co.行业=='先进制造')]

df_patent_invest=pd.read_csv('data\\patent_invest_202407_发明授权_发明申请.csv',index_col=0)#patent_invest_202408_先进制造
df_patent_invest['融资时间']=pd.to_datetime(df_patent_invest['融资时间'])
final_invest_data_tech_1=df_patent_invest[(df_patent_invest.行业=='先进制造')].sort_values(by=['公司序号','融资时间'])#&(final_invest_data_tech.上市Days.isna())
final_invest_data_tech_1.shape

In [24]:
doc_lda_w2v_1.shape,patent_co_1.shape

((1348153, 250), (1203317, 39))

In [18]:
doc_lda_w2v_1.index.value_counts()

序号
CN106370313B    4
CN104183435B    4
CN204067280U    4
CN204067282U    4
CN204067283U    4
               ..
CN105244535B    1
CN105239082B    1
CN105226318B    1
CN105244543B    1
CN3044990D      1
Name: count, Length: 1171534, dtype: int64

In [32]:
doc_lda_w2v=doc_lda_w2v_1.groupby(doc_lda_w2v_1.index).mean()
doc_lda_w2v.shape

(1171534, 250)

In [37]:
set(doc_lda_w2v.index)-set(patent_co_1['公开（公告）号'])

set()

In [7]:
final_invest_data_tech_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107145 entries, 20629 to 127770
Data columns (total 62 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   公司序号             107145 non-null  int64         
 1   公司简称             107145 non-null  object        
 2   工商全称             106926 non-null  object        
 3   成立时间             107145 non-null  object        
 4   公司简介             107145 non-null  object        
 5   省                107145 non-null  object        
 6   市                107145 non-null  object        
 7   行业               107145 non-null  object        
 8   子行业              107145 non-null  object        
 9   估算估值             99741 non-null   float64       
 10  北上广深             107145 non-null  int64         
 11  IPO上市日期          26292 non-null   object        
 12  首次上市日期           24372 non-null   object        
 13  目前已上市            107145 non-null  int64         
 14  高科技企业            1071

In [35]:
final_invest_data_tech_1['观察日']=np.where(final_invest_data_tech_1.融资时间.notna(), final_invest_data_tech_1.融资时间,
                                         pd.to_datetime(final_invest_data_tech_1['观察年'], format='%Y')+ pd.offsets.YearEnd())
final_invest_data_tech_1['观察日'].isna().sum()

0

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
tech_category

1721380    F21
1739616    F21
1739617    H05
Name: IPC主分类, dtype: object

In [65]:
 np.empty(len(base_category))

array([0., 0., 0., 0.])

In [67]:
patent_co_1['技术大类']=patent_co_1.IPC主分类.str.strip().str[:3]

C:\Users\Prime\AppData\Local\Temp\ipykernel_24800\1958649803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_co_1['技术大类']=patent_co_1.IPC主分类.str.strip().str[:3]


In [ ]:
%%time
tuple_list = [(1, 4), (2, 4), (1, 5), (2, 5), (2, 6), (3, 4), (3, 5)]
Threshold=0.8
Year=None
for date in sorted(final_invest_data_tech_1['观察日'].unique()):
    if date.year!=Year:
        print(date.year)
        Year=date.year
        
    fiter_invest=final_invest_data_tech_1[final_invest_data_tech_1['观察日']==date]
    filter_value=patent_co_1[patent_co_1.申请日<date]
    if filter_value.empty:
        continue
    
    for index,row in fiter_invest.iterrows():
        filter_value_0=filter_value[filter_value.工商全称==row.工商全称]
        if filter_value_0.empty:
            continue
            
        metrics = {}
        for m,n in tuple_list:
            suffix=f'{m,n}'
            M=m*365
            N=n*365
            filter_value_1=filter_value_0[filter_value_0.申请日>=date-pd.Timedelta(days=N)]
            if filter_value_1.empty:
                continue
                
            mask=filter_value_1.申请日 <  date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_1[mask]
            base_category_value = filter_value_1[~mask]
            
            tech_category= doc_lda_w2v.loc[doc_lda_w2v.index.isin(tech_category_value['公开（公告）号'])]
            base_category= doc_lda_w2v.loc[doc_lda_w2v.index.isin(base_category_value['公开（公告）号'])]
            if not tech_category.empty and not base_category.empty:
                cosine_sim = cosine_similarity(tech_category.values,base_category.values)#shape:len(tech_category) x len(base_category)
                tech_base_i = cosine_sim.mean(axis=0) #len(base_category)
                innovation_patent_num = (tech_base_i < Threshold).sum() #统计base_category中小于threshold的专利个数
                
                metrics[f'innovation_patent_num_{suffix}']=innovation_patent_num
                metrics[f'tech_closeness_innovation_{suffix}']=innovation_patent_num / len(filter_value_1)
                metrics[f'tech_base_i_mean_{suffix}']= tech_base_i.mean()      
    
        #--------
            diff_ipcs=set(tech_category_value.技术大类)^set(base_category_value.技术大类)
            innovation_patent_num=len(diff_ipcs)
            
            metrics[f'innovation_patent_num_ipc_{suffix}']=innovation_patent_num
            metrics[f'ipc_innovationc_{suffix}']=innovation_patent_num / len(filter_value_1)
           
            metrics[f'overlap_patent_num_{suffix}'] = len(set(base_category.index) & set(filter_value_1.loc[filter_value_1.技术大类.isin(diff_ipcs),'公开（公告）号']))# 计算 tech_base_i 和 IPC 分类差集的重合部分
    
            
        final_invest_data_tech_1.loc[index, metrics.keys()] = pd.Series(metrics) 


    
final_invest_data_tech_1

1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


In [72]:
pd.Series(metrics) 

innovation_patent_num_(1, 4)        35589.000000
tech_closeness_innovation_(1, 4)        0.690003
tech_base_i_mean_(1, 4)                 0.731518
innovation_patent_num_ipc_(1, 4)       25.000000
ipc_innovationc_(1, 4)                  0.000485
overlap_patent_num_(1, 4)             258.000000
innovation_patent_num_(2, 4)        19597.000000
tech_closeness_innovation_(2, 4)        0.379949
tech_base_i_mean_(2, 4)                 0.734651
innovation_patent_num_ipc_(2, 4)       18.000000
ipc_innovationc_(2, 4)                  0.000349
overlap_patent_num_(2, 4)              64.000000
innovation_patent_num_(1, 5)        49244.000000
tech_closeness_innovation_(1, 5)        0.760220
tech_base_i_mean_(1, 5)                 0.735229
innovation_patent_num_ipc_(1, 5)       20.000000
ipc_innovationc_(1, 5)                  0.000309
overlap_patent_num_(1, 5)             167.000000
innovation_patent_num_(2, 5)        35281.000000
tech_closeness_innovation_(2, 5)        0.544662
tech_base_i_mean_(2,

In [75]:
filter_value_0

,工商全称,序号,标题 (中文),摘要 (中文),申请人,申请日,申请号,首次公开日,公开（公告）号,专利类型,...,当前法律状态,table_name,公开类型,行业,申请年,首次上市日期,IPO上市日期,子行业,首次融资日期,技术大类
199506,珠海格力电器股份有限公司,381391,一种空调防直吹的控制方法及装置,本发明提供了一种空调防直吹的控制方法及装置，所述空调设置有上出风口和下出风口，所述上出风口设...,珠海格力电器股份有限公司,2022-12-30,CN202211720526.8,2023-04-07,CN115930405A,发明申请,...,公开,patent_incopat_202212_202312,NaN,先进制造,2022,1996-11-18,1996-11-18,家用电器,1996-11-18,F24
199510,珠海格力电器股份有限公司,397807,洗地机的自清洁控制方法、装置、存储介质和洗地机,本发明公开了一种洗地机的自清洁控制方法、装置、存储介质和洗地机，所述方法包括：在接收到洗地机...,珠海格力电器股份有限公司,2022-12-30,CN202211721134.3,2023-04-04,CN115886668A,发明申请,...,公开,patent_incopat_202212_202312,NaN,先进制造,2022,1996-11-18,1996-11-18,家用电器,1996-11-18,A47
199532,珠海格力电器股份有限公司,381390,空气调节器除霜控制方法、装置、空气调节器及存储介质,本申请涉及一种空气调节器除霜控制方法、装置、空气调节器、存储介质和计算机程序产品，获取空气调...,珠海格力电器股份有限公司; 珠海格力节能环保制冷技术研究中心有限公司,2022-12-30,CN202211723308.X,2023-04-07,CN115930366A,发明申请,...,公开,patent_incopat_202212_202312,NaN,先进制造,2022,1996-11-18,1996-11-18,家用电器,1996-11-18,F24
199566,珠海格力电器股份有限公司,388176,一种气浮电机系统,本发明提供一种气浮电机系统，其包括：机壳、转轴、电机组件、一级压缩组件、二级压缩组件、第一径...,珠海格力电器股份有限公司,2022-12-30,CN202211729267.5,2023-04-04,CN115912765A,发明申请,...,公开,patent_incopat_202212_202312,NaN,先进制造,2022,1996-11-18,1996-11-18,家用电器,1996-11-18,H02
199576,珠海格力电器股份有限公司,394153,一种机器人的散热组件和机器人,本发明提供一种机器人的散热组件和机器人，机器人的散热组件，其包括：J1减速器、J1电机、散热...,珠海格力电器股份有限公司,2022-12-30,CN202211731168.0,2023-04-04,CN115890746A,发明申请,...,公开,patent_incopat_202212_202312,NaN,先进制造,2022,1996-11-18,1996-11-18,家用电器,1996-11-18,B25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418446,珠海格力电器股份有限公司,371,空调器(移动式),使用状态立体图__使用状态参考图,珠海格力电器股份有限公司,1995-12-01,CN95319857.X,1996-12-25,CN3052899D,外观设计,...,未缴年费,patent202210,外观设计,先进制造,1995,1996-11-18,1996-11-18,家用电器,1996-11-18,NaN
2418459,珠海格力电器股份有限公司,188,电风扇,NaN,珠海格力电器股份有限公司,1994-09-16,CN94311570.1,1995-09-13,CN3035099D,外观设计,...,未缴年费,patent202210,外观设计,先进制造,1994,1996-11-18,1996-11-18,家用电器,1996-11-18,NaN
2418461,珠海格力电器股份有限公司,180,电风扇摇头装置,一种电风扇摇头装置，其特征在于由两个同轴安 装的上传动盘和下传动盘组成，上传动盘的下表面设 ...,珠海格力电器股份有限公司,1994-08-31,CN94219340.7,1995-07-05,CN2202815Y,实用新型,...,未缴年费,patent202210,实用新型,先进制造,1994,1996-11-18,1996-11-18,家用电器,1996-11-18,F04
2418462,珠海格力电器股份有限公司,181,变幅摇头电风扇,一种变幅摇头电风扇，包括有扇座、扇体，其特征 在于扇体变幅摇头机构由电机驱动的一个T形连 杆...,珠海格力电器股份有限公司,1994-08-31,CN94219339.3,1995-06-21,CN2201504Y,实用新型,...,未缴年费,patent202210,实用新型,先进制造,1994,1996-11-18,1996-11-18,家用电器,1996-11-18,F04


In [77]:
final_invest_data_tech_1['成立时间']=pd.to_datetime(final_invest_data_tech_1['成立时间'])
final_invest_data_tech_1['成立到首次融资']=(final_invest_data_tech_1.groupby('公司序号')['融资时间'].transform('min')-final_invest_data_tech_1['成立时间']).dt.days

In [78]:
final_invest_data_tech_1.to_excel('data/patent_innovation_202407_先进制造.xlsx')